# Blank and Bad Detector Pixels


**Learning how to deal with missing data and bad detector pixels.**

---

In [ ]:
import ptypy, os
import ptypy.utils as u

# This will import the HDF5Loader class
ptypy.load_ptyscan_module("hdf5_loader")

# This will import the GPU engines
ptypy.load_gpu_engines("cuda")  

# Root directory of tutorial data
tutorial_data_home = "/dls/science/groups/imaging/ptypy_tutorials/"

# Path to HDF5 file with raw data and positions
dataset = "dls_i13_butterfly/raw/excalibur_306517_vds.h5"
positions = "dls_i13_butterfly/processing/pos/306517.h5"

# Path to a previous reconstruction
init_probe = "dls_i13_butterfly/processing/ptypy/testing/303079_ML_pycuda_1500.ptyr"

# Path to HDF5 file with mask
mask = "dls_i13_butterfly/processing/masks/excalibur_512x512.h5"

# Absolute paths to HDF5 files with raw data, positions
# initial probe and mask
path_to_data = os.path.join(tutorial_data_home, dataset)
path_to_pos = os.path.join(tutorial_data_home, positions)
path_to_probe = os.path.join(tutorial_data_home, init_probe)
path_to_mask = os.path.join(tutorial_data_home, mask)

# Create parameter tree
p = u.Param()

# Set verbose level to info
p.verbose_level = "interactive"

# Scan label
p.run = "dls_i13_butterfly"

# Data loading and processing should 
# happen in chunks of this size
p.frames_per_block = 264

# Set io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Define the scan model
p.scans = u.Param()
p.scans.scan_00 = u.Param()
p.scans.scan_00.name = 'BlockFull'

# Initial illumination (based on previous reconstruction)
p.scans.scan_00.illumination = u.Param()
p.scans.scan_00.illumination.model = "recon"
p.scans.scan_00.illumination.recon = u.Param()
p.scans.scan_00.illumination.recon.rfile = path_to_probe
p.scans.scan_00.illumination.photons = None
p.scans.scan_00.illumination.aperture = u.Param()
p.scans.scan_00.illumination.aperture.form = None
p.scans.scan_00.illumination.diversity = u.Param()
p.scans.scan_00.illumination.diversity.power = 0.1
p.scans.scan_00.illumination.diversity.noise = [0.5,1.0]

# Initial object
p.scans.scan_00.sample = u.Param()
p.scans.scan_00.sample.model = None
p.scans.scan_00.sample.diversity = None
p.scans.scan_00.sample.process = None

# Coherence parameters (modes)
p.scans.scan_00.coherence = u.Param()
p.scans.scan_00.coherence.num_probe_modes = 1
p.scans.scan_00.coherence.num_object_modes = 1

# Data loader
p.scans.scan_00.data = u.Param()
p.scans.scan_00.data.name = 'Hdf5LoaderFast'
p.scans.scan_00.data.orientation = 0

p.scans.scan_00.data.intensities = u.Param()
p.scans.scan_00.data.intensities.file = path_to_data
p.scans.scan_00.data.intensities.key = "data"

p.scans.scan_00.data.positions = u.Param()
p.scans.scan_00.data.positions.file = path_to_pos
p.scans.scan_00.data.positions.slow_key = "slow"
p.scans.scan_00.data.positions.slow_multiplier = 1e-6
p.scans.scan_00.data.positions.fast_key = "fast"
p.scans.scan_00.data.positions.fast_multiplier = 1e-6

p.scans.scan_00.data.mask = u.Param()
p.scans.scan_00.data.mask.file = path_to_mask
p.scans.scan_00.data.mask.key = "data"

p.scans.scan_00.data.distance = 14.65
p.scans.scan_00.data.energy = 9.7
p.scans.scan_00.data.psize = 55e-6
p.scans.scan_00.data.auto_center = False
p.scans.scan_00.data.center = (903.5, 1018.)
p.scans.scan_00.data.shape = (512,512)
p.scans.scan_00.data.save = None

# Reconstruct using GPU-accelerated SDR
p.engines = u.Param()
p.engines.engine_dr = u.Param()
p.engines.engine_dr.name = "SDR_pycuda"
p.engines.engine_dr.numiter = 100
p.engines.engine_dr.numiter_contiguous = 1
p.engines.engine_dr.sigma = 0.5
p.engines.engine_dr.tau = 0.1
p.engines.engine_dr.compute_log_likelihood = True
p.engines.engine_dr.compute_exit_error = False
#p.engines.engine_dr.fft_lib = "cuda"

# Run reconstruction
P = ptypy.core.Ptycho(p,level=5)

# Display the results
u.plot_recon_from_ptycho(P, cropping=0.1, pmin=-0.2, pmax=0.2)